In [1]:
import pickle
from joblib import load
import pandas as pd

In [ ]:
model = load('T5-small-model.pkl')
tokenizer = load('T5-small.pkl')
device = load('device.pkl')

In [6]:
!pip install sentencepiece

  Using cached sentencepiece-0.2.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 254.4 kB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
df = pd.read_csv('1429_1.csv')
df = df[['asins','reviews.rating','reviews.text']]
df.dropna(inplace = True)
df = df.reset_index(drop=True)

C:\Users\Toshiba\AppData\Local\Temp\ipykernel_19144\133304360.py:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('1429_1.csv')


In [4]:
# Choose one product
product1 = df.where(df['asins'] == 'B01AHB9CN2')
product1 = product1.dropna()
product1 = product1.reset_index(drop=True)

In [5]:
# split reviews into pos & neg
pos_rev = ''
neg_rev = ''
for i in range(len(product1)):
    if df['reviews.rating'][i] > 3.0:
        pos_rev = pos_rev + (str(df['reviews.text'][i]))
    elif df['reviews.rating'][i] <= 3.0:
        neg_rev = neg_rev + (str(df['reviews.text'][i]))

In [6]:
def summarize(pos_rev, neg_rev):
    # preprocess the pos&neg input text
    pos_preprocessed_text = pos_rev.strip().replace('\n','')
    pos_t5_input_text = 'summarize: ' + pos_rev
    neg_preprocessed_text = neg_rev.strip().replace('\n','')
    neg_t5_input_text = 'summarize: ' + neg_rev

    pos_tokenized_text = tokenizer.encode(pos_t5_input_text, return_tensors='pt', max_length=512).to(device)
    neg_tokenized_text = tokenizer.encode(neg_t5_input_text, return_tensors='pt', max_length=512).to(device)

    # summarize
    pos_summary_ids = model.generate(pos_tokenized_text, min_length=30, max_length=120)
    pos_summary = tokenizer.decode(pos_summary_ids[0], skip_special_tokens=True)

    neg_summary_ids = model.generate(neg_tokenized_text, min_length=30, max_length=120)
    neg_summary = tokenizer.decode(neg_summary_ids[0], skip_special_tokens=True)

    return pos_summary, neg_summary

In [7]:
pos_summary, neg_summary = summarize(pos_rev, neg_rev)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [8]:
pos_summary

'amazon fire 8 inch tablet is the perfect size for e-reading on the go. great for e-reading on the go, nice and light weight, and definitely worth the purchase.'

In [9]:
neg_summary

'i bought this tablet for my 4 year old daughter. it is too tied to amazon. It requires too many hoops to get apps to work.'